# Additional Modelling Challenges

## Balanced Nurse Assignment

Given a set of patients distributed in a number of hospital zones and an available nursing staff, one must assign a nurse to each patient in such a way that the work is distributed evenly between nurses.

1. Each patient is assigned an acuity level corresponding to the amount of care he requires.
2. The workload of a nurse is defined as the sum of the acuities of the patients he cares for.
3. A nurse can only work in one zone.

This problem has two connected aspects:

- Nurses need to be assigned to zones.
- Patients of that zone have to be assigned to nurses.

Balance the workloads by minimizing their standard deviation.

In [1]:
import cpmpy as cp
import random

random.seed(42)

# Parameters
n_nurses = 5
n_patients = 25
n_zones = 3

# 1. Each patient is assigned an acuity level corresponding to the amount of care he requires.
patients_acuity_levels = random.choices([1, 2, 3, 4, 5], k=n_patients)
patients_zones = [i % n_zones for i in range(n_patients)]

# Decision variables
patients_nurses = cp.intvar(0, n_nurses - 1, shape=n_patients, name="patients_nurses")  # Nurses assigned to patients
nurses_zones = cp.intvar(0, n_zones - 1, shape=n_nurses, name="nurses_zones")  # Zones assigned to nurses

# Workload variables
nurses_workloads = cp.intvar(0, sum(patients_acuity_levels), shape=n_nurses, name="nurses_workloads")

model = cp.Model()

# 2. The workload of a nurse is defined as the sum of the acuities of the patients he cares for.
for n in range(n_nurses):
    model += nurses_workloads[n] == cp.sum(
        [patients_acuity_levels[p] * (patients_nurses[p] == n) for p in range(n_patients)]
    )

# 3. A nurse can only work in one zone
for p in range(n_patients):
    model += (nurses_zones[patients_nurses[p]] == patients_zones[p])

# Balance the workloads by minimizing their variance
mean_workload = cp.sum(nurses_workloads) // n_nurses
variance = cp.sum((nurses_workloads - mean_workload)**2) // n_nurses
model.minimize(variance)

# Solve the model
if model.solve():
    print("Solution found!")
    print("Patients' acuity levels:", patients_acuity_levels)
    print("Patients' nurses assignments:", patients_nurses.value())
    print("Nurses' zones assignments:", nurses_zones.value())
    print("Nurses' workloads:", [nurses_workloads[n].value() for n in range(n_nurses)])
else:
    print("No solution found.")

Solution found!
Patients' acuity levels: [4, 1, 2, 2, 4, 4, 5, 1, 3, 1, 2, 3, 1, 1, 4, 3, 2, 3, 5, 1, 5, 4, 2, 1, 5]
Patients' nurses assignments: [3 1 0 2 1 4 3 1 0 3 1 4 3 1 0 2 1 0 2 1 4 3 1 4 2]
Nurses' zones assignments: [2 1 0 0 2]
Nurses' workloads: [12, 14, 15, 15, 13]


In [2]:
nurses_workload = 0
for nurse in range(n_nurses):
    nurse_patients = [idx for idx, value in enumerate(patients_nurses.value()) if value == nurse]
    nurse_workloads = [patients_acuity_levels[p] for p in nurse_patients]
    nurse_workload = sum(nurse_workloads)
    nurses_workload += nurse_workload
    nurse_zone = nurses_zones[nurse].value()

    print(nurse, nurse_zone, nurse_workloads, nurse_workload)
assert nurses_workload == sum(patients_acuity_levels)
print(nurses_workload)

0 2 [2, 3, 4, 3] 12
1 1 [1, 4, 1, 2, 1, 2, 1, 2] 14
2 0 [2, 3, 5, 5] 15
3 0 [4, 5, 1, 1, 4] 15
4 2 [4, 3, 5, 1] 13
69
